<a href="https://colab.research.google.com/github/Mahnazshamissa/Python/blob/main/asigmo_ex3_VGG19_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab exercise 3


Per Group:
- 1.   Create a teams-meeting where one person opens the colab notebook and shares the screen (preferably a different person than yesterday)
2.   Don't unfreeze the pretrained ResNet50 and train it together with the last layer from scratch on 20 epochs. What's the best validation accuracy and test accuracy you get?
3.   Pretrain the output layer on 20 epochs, then unfreeze all layers and train it for 10 epochs. What's the best validation accuracy and test accuracy you get?
4.   Pretrain the output layer on 20 epochs, then unfreeze only the first 10 layers and train it for 10 epochs. What's the best validation accuracy and test accuracy you get?
5.   Think about why your network performed better or worse compared to the ~94% accuracy of the baseline network
6.   Change the ResNet50 backbone to a ```EfficientNetB4```(group A and C) or ```VGG19``` (group D and E) and train only the output layer. What's the best validation accuracy and test accuracy you get? (don't forget to also change the preprocessing function)
7.   In each group channel answer the questions below

**Warning:** In steps 3 to 5 make sure you overwrite the model and don't continue training on the model from the previous step!


**Questions**
1.  Assuming a ResNet50 training step with a batchsize of 64 takes around 900ms, how long would one training epoch take on ImageNet (1.3 million training images)?
2. How long would it then take to train the ResNet50 for 100 epochs?

In [ ]:
imageNetSize =1300000
imageNetBatches = imageNetSize / 64
timePerBatch = 0.9 #seconds
totalTime = imageNetBatches * timePerBatch
timePerEpochInHours = totalTime/(60*60)
print("time to run one epoch with all images in imageNet: %f hours"%timePerEpochInHours)
totalTimeTraining = timePerEpochInHours * 100 / 24
print("time to run 100 epocsh with all images in imageNet: %f days"%totalTimeTraining)

time to run one epoch with all images in imageNet: 5.078125 hours
time to run 100 epocsh with all images in imageNet: 21.158854 days


## Setup notebook environment

Make sure the Colab environment has a GPU enabled *Edit->Notebook Settings->Hardware accelerator->Choose GPU*

In [ ]:
! nvidia-smi

Thu Oct 29 10:24:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Download the dataset first

In [ ]:
! wget https://pub.ist.ac.at/~mlechner/datasets/f8d.tar.gz
! tar -xf f8d.tar.gz

--2020-10-29 10:24:58--  https://pub.ist.ac.at/~mlechner/datasets/f8d.tar.gz
Resolving pub.ist.ac.at (pub.ist.ac.at)... 81.223.84.195
Connecting to pub.ist.ac.at (pub.ist.ac.at)|81.223.84.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76149807 (73M) [application/x-gzip]
Saving to: ‘f8d.tar.gz’

f8d.tar.gz          100%[===================>]  72.62M  6.09MB/s    in 11s     

2020-10-29 10:25:09 (6.61 MB/s) - ‘f8d.tar.gz’ saved [76149807/76149807]



Import python libraries

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## Step 1: Define datasets

In [ ]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  "f8d/test", shuffle=False, batch_size=128, image_size=(256, 256),
)
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  "f8d/training",
  shuffle=True,
  batch_size=64,
  image_size=(256, 256),
  validation_split=0.2,
  subset="training",
  seed=1020202,
)
valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  "f8d/training",
  shuffle=True,
  batch_size=128,
  image_size=(256, 256),
  validation_split=0.2,
  subset="validation",
  seed=1020202,
)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
valid_dataset = valid_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

Found 640 files belonging to 8 classes.
Found 1840 files belonging to 8 classes.
Using 1472 files for training.
Found 1840 files belonging to 8 classes.
Using 368 files for validation.


## Step 2 and 3: Define model and objective

In [ ]:
pretrained_model = tf.keras.applications.VGG19(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(256,256,3)),
  tf.keras.layers.experimental.preprocessing.RandomCrop(224, 224),
  tf.keras.layers.Lambda(lambda x: tf.keras.applications.vgg19.preprocess_input(x)),
  pretrained_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(8, activation="softmax"),
])
pretrained_model.trainable=False
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_crop_2 (RandomCrop)   (None, 224, 224, 3)       0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 4104      
Total params: 20,028,488
Trainable params: 4,104
Non-trainable params: 20,024,384
_________________________________________________________________


## Step 4: Train the last layer

In [ ]:
training_log = model.fit(
  train_dataset, epochs=20, validation_data=valid_dataset
)

Epoch 1/20
23/23 [==============================] - 8s 368ms/step - loss: 4.9241 - sparse_categorical_accuracy: 0.3390 - val_loss: 2.5943 - val_sparse_categorical_accuracy: 0.5109
Epoch 2/20
23/23 [==============================] - 5s 198ms/step - loss: 1.8499 - sparse_categorical_accuracy: 0.6345 - val_loss: 1.4875 - val_sparse_categorical_accuracy: 0.6821
Epoch 3/20
23/23 [==============================] - 5s 199ms/step - loss: 1.0492 - sparse_categorical_accuracy: 0.7548 - val_loss: 1.1712 - val_sparse_categorical_accuracy: 0.7418
Epoch 4/20
23/23 [==============================] - 5s 199ms/step - loss: 0.7363 - sparse_categorical_accuracy: 0.8186 - val_loss: 0.9613 - val_sparse_categorical_accuracy: 0.7853
Epoch 5/20
23/23 [==============================] - 5s 201ms/step - loss: 0.5503 - sparse_categorical_accuracy: 0.8567 - val_loss: 0.8919 - val_sparse_categorical_accuracy: 0.8125
Epoch 6/20
23/23 [==============================] - 5s 200ms/step - loss: 0.4153 - sparse_categorica

In [ ]:
print(f"Best validation accuracy is {100*np.max(training_log.history['val_sparse_categorical_accuracy']):0.2f}%")

Best validation accuracy is 86.41%


## Step 5: Evaluate the model

In [ ]:
model.evaluate(test_dataset)

5/5 [==============================] - 1s 228ms/step - loss: 0.6627 - sparse_categorical_accuracy: 0.8391


[0.662731945514679, 0.839062511920929]

## Unfreeze the weights of the last 10 layers

In [ ]:
pretrained_model.trainable = True
for l in pretrained_model.layers:
  l.trainable=False
for i in range(10):
  pretrained_model.layers[-i].trainable=True
  
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(0.0001),
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

In [ ]:
training_log2 = model.fit(
  train_dataset, epochs=10, validation_data=valid_dataset
)

In [ ]:
model.evaluate(test_dataset)